In [ ]:
!python -m pip install -e . >> /dev/null

In [ ]:
import toolviper
import xradio
import pathlib
import numba

import numpy as np
import pandas as pd
import calviper as cv
import matplotlib.pyplot as plt

from xradio import measurement_set as ms

In [ ]:
if not pathlib.Path.cwd().joinpath("data/gaincaltest2.ps.zarr").exists():
    toolviper.utils.data.download("gaincal.test.zarr", "data")

In [ ]:
ps = ms.open_processing_set("data/gaincaltest2.ps.zarr")

sub_ps = ps.sel(intents="CALIBRATE_DELAY#ON_SOURCE", scan_number=2)
sub_ps.summary()

In [ ]:
dataset = sub_ps["gaincaltest2_0"]

In [ ]:
V = dataset.VISIBILITY.mean(dim="time").data.compute()
V.shape

In [ ]:
s = dataset.VISIBILITY.shape
s

In [ ]:
gain_jones = cv.jones.GainJones()

In [ ]:
G = gain_jones.from_visibility(dataset=dataset)

In [ ]:
G.matrix.shape

In [ ]:
G.parameters.shape

In [ ]:
g = G.matrix.mean(axis=0)
g.shape

In [ ]:
# This should all be done within the equivalent of VisEquation
#v = V[:, 0, 0]
v = V

index_a, ant = cv.math.tools.encode(dataset.baseline_antenna1_name.to_numpy())
index_b, _ = cv.math.tools.encode(dataset.baseline_antenna2_name.to_numpy())

v_ = cv.math.tools.build_visibility_matrix(array=v, index_a=index_a, index_b=index_b)

In [ ]:
v_.shape

In [ ]:
solver = cv.math.solver.least_squares.LeastSquaresSolver()

In [ ]:
gain_solutions = solver.solve(
    vis=v_,
    iterations=50,
    optimizer=cv.math.optimizer.MeanSquaredError(alpha=0.25),
    stopping=1e-4
)

In [ ]:
t = np.linspace(1, len(solver.losses), len(solver.losses))

plt.scatter(solver.losses, t)

In [ ]:
# Gain solutions

solver.parameter

In [ ]:
solver.parameter.shape